In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from tensorflow.keras.layers import GlobalAveragePooling1D, Concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [35]:
from tqdm.auto import tqdm
from nltk.tokenize import sent_tokenize, word_tokenize

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = "/content/drive/My Drive/Capstone/data/depression_pre.csv"
dp = pd.read_csv(path)

In [5]:
path2 = "/content/drive/My Drive/Capstone/data/depression_post.csv"
dp_post = pd.read_csv(path2)

In [ ]:
pip install --upgrade transformers

In [ ]:
pip install --upgrade tensorflow

In [ ]:
pip install --upgrade bert-for-tf2

In [6]:
pre = dp.drop(columns=['Unnamed: 0', 'Post URL', 'Original Content', 'Saved','ID', 'year', 'month'])
pre

,Title,Post Text,Score,Upvote Ratio,Total Comments,Created On
0,"Anyone else feel like their ""fake depressed""?","it's stupid, but i always doubt the fact i hav...",2195,0.99,156,2018-01-28 13:12:11
1,Why don't you talk to us?,My parents keep asking me this question.\nToda...,1652,0.99,136,2018-01-20 17:05:18
2,An old man killed my only way of coping with d...,- be me\n- have chronic depression and social ...,1644,0.98,107,2018-01-26 20:18:05
3,Anyone just want to take a break from life and...,"That's all I want. I wanna go to a park, lay o...",1403,0.99,104,2018-01-17 00:06:20
4,I had to fill out one of those depression test...,And I scored in the 'severe' range. This lead ...,1299,0.99,96,2018-01-08 13:05:09
...,...,...,...,...,...,...
322,Why does depression and anxiety make it so fuc...,Seriously- I can’t tell even if I’ve known the...,1414,1.00,74,2020-01-12 00:23:46
323,Does anyone ever feel physically too weak. Eve...,I’m beginning to wonder if I have a medical co...,1251,1.00,103,2020-02-01 15:29:40
324,It’s not fair.,"I fucking hate it so much. People around me, j...",1222,0.99,121,2020-01-23 13:13:37
325,I’m tired of being tired all the time.,I’m lying in bed exhausted. Tried to do homewo...,1150,1.00,83,2020-01-18 05:11:20


In [ ]:
post = dp_post.drop(columns=['Unnamed: 0', 'Post URL', 'Original Content', 'Saved','ID', 'year', 'month'])
post

In [8]:
pre_txt = pre[["Post Text"]]
post_txt = post[["Post Text"]]

In [9]:
pre_txt['label'] = 'pre'
post_txt['label'] = 'post'

<ipython-input-9-26faaad642e7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_txt['label'] = 'pre'
<ipython-input-9-26faaad642e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_txt['label'] = 'post'


In [10]:
pre_txt['Post Text'] = pre_txt['Post Text'].values
post_txt['Post Text'] = post_txt['Post Text'].values

<ipython-input-10-714712031f77>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_txt['Post Text'] = pre_txt['Post Text'].values
<ipython-input-10-714712031f77>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_txt['Post Text'] = post_txt['Post Text'].values


In [11]:
pre_txt['Post Text'].fillna('', inplace=True)
pre_txt.dropna(subset=['Post Text'], inplace=True)

<ipython-input-11-7f9de0dda555>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_txt['Post Text'].fillna('', inplace=True)
<ipython-input-11-7f9de0dda555>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_txt.dropna(subset=['Post Text'], inplace=True)


In [12]:
post_txt['Post Text'].fillna('', inplace=True)
post_txt.dropna(subset=['Post Text'], inplace=True)

<ipython-input-12-60cd906aa43a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_txt['Post Text'].fillna('', inplace=True)
<ipython-input-12-60cd906aa43a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_txt.dropna(subset=['Post Text'], inplace=True)


In [13]:
pre_txt

,Post Text,label
0,"it's stupid, but i always doubt the fact i hav...",pre
1,My parents keep asking me this question.\nToda...,pre
2,- be me\n- have chronic depression and social ...,pre
3,"That's all I want. I wanna go to a park, lay o...",pre
4,And I scored in the 'severe' range. This lead ...,pre
...,...,...
322,Seriously- I can’t tell even if I’ve known the...,pre
323,I’m beginning to wonder if I have a medical co...,pre
324,"I fucking hate it so much. People around me, j...",pre
325,I’m lying in bed exhausted. Tried to do homewo...,pre


In [14]:
pre_txt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Post Text  327 non-null    object
 1   label      327 non-null    object
dtypes: object(2)
memory usage: 5.2+ KB


In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [16]:
pre_txt['Post Text'].iloc[0]

'it\'s stupid, but i always doubt the fact i have depression, even after getting an official diagnosis, i\'m still scared that i\'m just an attention whore taking up everyone\'s time. Most of the time when i go to post something on here, i end up deleting it because i don\'t want someone in the comments to say something like "your not actually depressed, your just a whiny teen". I don\'t know. \n\nEDIT: Hey so i\'m not the only one who feels this way. If anything it\'s slightly comforting to know that\n\nEDIT 2: Yes i\'m aware of the spelling mistake in the title, it\'s kinda hard to spell correctly when you\'re rambling at 11 pm'

In [18]:
token = tokenizer.encode_plus(
    pre_txt['Post Text'].iloc[0],
    max_length = 256,
    padding='max_length',
    truncation=True,
    add_special_tokens=True,
    return_tensors='tf'
)

In [25]:
X_input_ids = np.zeros((len(pre_txt), 256))
X_attn_mask = np.zeros((len(pre_txt), 256))

In [26]:
X_input_ids.shape

(327, 256)

In [36]:
def generate_training_data(pre_txt, ids, masks, tokenizer):
  for i, text in tqdm(enumerate(pre_txt['Post Text'])):
      tokenized_text = tokenizer.encode_plus(
        text,
        max_length=256,
        padding='max_length',
        truncation=True,
        add_special_tokens=True,
        return_tensors='tf'
      )
      ids[i, :] = tokenized_text.input_ids
      masks[i, :] = tokenized_text.attention_mask
  return ids, masks

In [37]:
X_input_ids, X_attn_mask = generate_training_data(pre_txt, X_input_ids, X_attn_mask, tokenizer)

0it [00:00, ?it/s]

In [38]:
labels = np.zeros((len(pre_txt), 5))
labels.shape

(327, 5)

In [ ]:
# pre covid
input_ids_before = []
attention_masks_before = []

In [ ]:
for text in pre_txt['Post Text']:
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )

    input_ids_before.append(encoded_dict['input_ids'])
    attention_masks_before.append(encoded_dict['attention_mask'])

input_ids_before = tf.concat(input_ids_before, axis=0)
attention_masks_before = tf.concat(attention_masks_before, axis=0)

In [ ]:
input_ids_after = []
attention_masks_after = []

In [ ]:
for text in post_txt['Post Text']:
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )

    input_ids_after.append(encoded_dict['input_ids'])
    attention_masks_after.append(encoded_dict['attention_mask'])

input_ids_after = tf.concat(input_ids_after, axis=0)
attention_masks_after = tf.concat(attention_masks_after, axis=0)

In [ ]:
# Convert labels to one-hot encoding
labels_before = pd.get_dummies(pre_txt['label']).values

labels_after = pd.get_dummies(post_txt['label']).values

In [ ]:
labels_before_flat = labels_before.flatten()
labels_after_flat = labels_after.flatten()

In [ ]:
input_ids_before_np = input_ids_before.numpy()
attention_masks_before_np = attention_masks_before.numpy()
labels_before_flat_np = labels_before_flat.astype(np.float32)  # Convert labels to float32

input_ids_after_np = input_ids_after.numpy()
attention_masks_after_np = attention_masks_after.numpy()
labels_after_flat_np = labels_after_flat.astype(np.float32)

In [ ]:
# Reshape labels for before COVID
labels_before_flat_np_reshaped = labels_before_flat_np.reshape(-1, 1)

# Reshape labels for after COVID
labels_after_flat_np_reshaped = labels_after_flat_np.reshape(-1, 1)

In [ ]:
train_inputs_before, test_inputs_before, train_masks_before, test_masks_before, train_labels_before, test_labels_before = train_test_split(
    input_ids_before_np,
    attention_masks_before_np,
    labels_before_flat_np_reshaped,
    test_size=0.2, random_state=42
)

# Split the datasets into training and testing sets for after COVID
train_inputs_after, test_inputs_after, train_masks_after, test_masks_after, train_labels_after, test_labels_after = train_test_split(
    input_ids_after_np,
    attention_masks_after_np,
    labels_after_flat_np_reshaped,
    test_size=0.2, random_state=42
)

In [ ]:
# Define the BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Freeze BERT layers
bert_model.trainable = False

# BERT inputs
input_ids = Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
attention_masks = Input(shape=(max_length,), dtype=tf.int32, name="attention_masks")

# BERT output
bert_output = bert_model([input_ids, attention_masks])[1]

# Dense layer
dense = Dense(64, activation='relu')(bert_output)

# Output layer
output = Dense(2, activation='softmax')(dense)

# Build model
model = Model(inputs=[input_ids, attention_masks], outputs=output)

# Compile model
optimizer = Adam(lr=2e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train model for before COVID
history_before = model.fit(
    [train_inputs_before, train_masks_before],
    train_labels_before,
    epochs=3,
    batch_size=32,
    validation_data=([test_inputs_before, test_masks_before], test_labels_before)
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

ValueError: Exception encountered when calling layer 'tf_bert_model_6' (type TFBertModel).

Data of type <class 'keras.src.engine.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for input_ids.

Call arguments received by layer 'tf_bert_model_6' (type TFBertModel):
  • input_ids=["<KerasTensor: shape=(None, 128) dtype=int32 (created by layer 'input_ids')>", "<KerasTensor: shape=(None, 128) dtype=int32 (created by layer 'attention_masks')>"]
  • attention_mask=None
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False

In [ ]:
print("Shapes before COVID:")
print("Input IDs:", input_ids_before.shape)
print("Attention Masks:", attention_masks_before.shape)
print("Labels:", labels_before_flat.shape)

print("Shapes after COVID:")
print("Input IDs:", input_ids_after.shape)
print("Attention Masks:", attention_masks_after.shape)
print("Labels:", labels_after_flat.shape)

Shapes before COVID:
Input IDs: (327, 128)
Attention Masks: (327, 128)
Labels: (327,)
Shapes after COVID:
Input IDs: (473, 128)
Attention Masks: (473, 128)
Labels: (473,)


In [ ]:
labels_before = pd.get_dummies(pre_txt['label']).values
labels_after = pd.get_dummies(post_txt['label']).values

In [ ]:
labels_before_flat = labels_before.flatten()

In [ ]:
train_inputs_before, test_inputs_before, train_masks_before, test_masks_before, train_labels_before, test_labels_before = train_test_split(
    input_ids_before,
    attention_masks_before,
    labels_before_flat,  # Use flattened labels
    test_size=0.2, random_state=42
)

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([183,  17,  24, 132, 113, 229,  75, 218, 140, 195, 108, 116, 126,
       109, 260,  31, 221, 148, 158, 225, 316, 193,  59, 269,   6, 202,
        30,  22, 278, 253,  56, 222, 137, 210,  81, 194,  39, 226, 104,
       152, 274, 155, 227, 143,  66, 203,  18, 289, 290,  19, 164,  92,
        79, 219, 157,  72,  96,  15, 111,  10, 185,  69,  97, 165,  68,
        23,  37, 237,  67, 262, 208, 125, 182,  86, 312, 313, 277, 145,
       294, 147, 268, 239, 211, 129,  38, 228, 197, 307, 112, 196, 117,
       302, 315, 296, 180, 265, 154, 285,   2, 115, 168, 206, 120, 249,
       283, 127,  74,  29,  83, 271, 107, 234, 184, 133, 223, 255, 301,
        65, 232,  85, 186, 305, 159,  12,  35,  28, 170, 142, 224, 131,
       198,  51,  95, 245, 178, 123,  41,  89, 256, 136,  26, 320, 141,
       282,   0, 310, 280, 297, 100, 192, 281, 213, 103, 171,  98,  36

In [ ]:
# Split the datasets into training and testing sets for before COVID
train_inputs_before, test_inputs_before, train_masks_before, test_masks_before, train_labels_before, test_labels_before = train_test_split(
    input_ids_before,
    attention_masks_before,
    labels_before,
    test_size=0.2, random_state=42
)

# Split the datasets into training and testing sets for after COVID
train_inputs_after, test_inputs_after, train_masks_after, test_masks_after, train_labels_after, test_labels_after = train_test_split(
    input_ids_after,
    attention_masks_after,
    labels_after,
    test_size=0.2, random_state=42
)

# Define the BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Freeze BERT layers
bert_model.trainable = False

# BERT inputs
input_ids = Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
attention_masks = Input(shape=(max_length,), dtype=tf.int32, name="attention_masks")

# BERT output
bert_output = bert_model([input_ids, attention_masks])[1]

# Dense layer
dense = Dense(64, activation='relu')(bert_output)

# Output layer
output = Dense(2, activation='softmax')(dense)

# Build model
model = Model(inputs=[input_ids, attention_masks], outputs=output)

# Compile model
optimizer = Adam(lr=2e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train model for before COVID
history_before = model.fit(
    [train_inputs_before, train_masks_before],
    train_labels_before,
    epochs=3,
    batch_size=32,
    validation_data=([test_inputs_before, test_masks_before], test_labels_before)
)

# Evaluate model for before COVID
predictions_before = model.predict([test_inputs_before, test_masks_before])
predicted_labels_before = np.argmax(predictions_before, axis=1)
true_labels_before = np.argmax(test_labels_before, axis=1)

print("Evaluation Report for Before COVID:")
print(classification_report(true_labels_before, predicted_labels_before))

# Train model for after COVID
history_after = model.fit(
    [train_inputs_after, train_masks_after],
    train_labels_after,
    epochs=3,
    batch_size=32,
    validation_data=([test_inputs_after, test_masks_after], test_labels_after)
)

# Evaluate model for after COVID
predictions_after = model.predict([test_inputs_after, test_masks_after])
predicted_labels_after = np.argmax(predictions_after, axis=1)
true_labels_after = np.argmax(test_labels_after, axis=1)

print("Evaluation Report for After COVID:")
print(classification_report(true_labels_after, predicted_labels_after))


TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([183,  17,  24, 132, 113, 229,  75, 218, 140, 195, 108, 116, 126,
       109, 260,  31, 221, 148, 158, 225, 316, 193,  59, 269,   6, 202,
        30,  22, 278, 253,  56, 222, 137, 210,  81, 194,  39, 226, 104,
       152, 274, 155, 227, 143,  66, 203,  18, 289, 290,  19, 164,  92,
        79, 219, 157,  72,  96,  15, 111,  10, 185,  69,  97, 165,  68,
        23,  37, 237,  67, 262, 208, 125, 182,  86, 312, 313, 277, 145,
       294, 147, 268, 239, 211, 129,  38, 228, 197, 307, 112, 196, 117,
       302, 315, 296, 180, 265, 154, 285,   2, 115, 168, 206, 120, 249,
       283, 127,  74,  29,  83, 271, 107, 234, 184, 133, 223, 255, 301,
        65, 232,  85, 186, 305, 159,  12,  35,  28, 170, 142, 224, 131,
       198,  51,  95, 245, 178, 123,  41,  89, 256, 136,  26, 320, 141,
       282,   0, 310, 280, 297, 100, 192, 281, 213, 103, 171,  98,  36

In [ ]:
# Evaluate model for before COVID
predictions_before = model.predict([test_inputs_before, test_masks_before])
predicted_labels_before = np.argmax(predictions_before, axis=1)
true_labels_before = np.argmax(test_labels_before, axis=1)

print("Evaluation Report for Before COVID:")
print(classification_report(true_labels_before, predicted_labels_before))


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Find the maximum sequence length in the "after" dataset
max_length_after = max(len(sequence) for sequence in input_ids_after)

# Pad sequences in the "before" dataset to match the maximum length in the "after" dataset
input_ids_before_padded = pad_sequences(input_ids_before, maxlen=max_length_after, padding='post')
attention_masks_before_padded = pad_sequences(attention_masks_before, maxlen=max_length_after, padding='post')


In [ ]:
max_length = max_length_after  # Use the maximum length from the "after" dataset

# Assuming you have labels for your datasets
# Convert labels to one-hot encoding
labels_before_one_hot = to_categorical(labels_before)
labels_after_one_hot = to_categorical(labels_after)

In [ ]:
min_samples = min(len(pre_txt), len(post_txt))

# Sample randomly from the larger dataset to match the size of the smaller one
pre_sampled = pre_txt.sample(n=min_samples, random_state=42)
post_covid_sampled = post_txt.sample(n=min_samples, random_state=42)


In [ ]:
train_before, test_before = train_test_split(pre_txt, test_size=0.2, random_state=42)

# Split "after" dataset into training and testing sets
train_after, test_after = train_test_split(post_txt, test_size=0.2, random_state=42)


In [ ]:
train_combined = pd.concat([train_before, train_after])
test_combined = pd.concat([test_before, test_after])

# Shuffle the combined datasets
train_combined = train_combined.sample(frac=1, random_state=42).reset_index(drop=True)
test_combined = test_combined.sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
# Define the BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Freeze BERT layers
bert_model.trainable = False

# BERT before inputs
input_ids_before = Input(shape=(max_length,), dtype=tf.int32, name="input_ids_before")
attention_masks_before = Input(shape=(max_length,), dtype=tf.int32, name="attention_masks_before")

# BERT after inputs
input_ids_after = Input(shape=(max_length,), dtype=tf.int32, name="input_ids_after")
attention_masks_after = Input(shape=(max_length,), dtype=tf.int32, name="attention_masks_after")


# BERT output before
bert_output_before = bert_model([input_ids_before, attention_masks_before])[0]

# BERT output after
bert_output_after = bert_model([input_ids_after, attention_masks_after])[0]

concatenated_output = Concatenate(axis=1)([bert_output_before, bert_output_after])

# pooling
pooled_output = GlobalAveragePooling1D()(concatenated_output)


# Dense layer
dense = Dense(64, activation='relu')(pooled_output)

# Output layer
output = Dense(2, activation='softmax')(dense)

# Build model
model = Model(inputs=[input_ids_before, attention_masks_before, input_ids_after, attention_masks_after], outputs=output)
# Compile model
optimizer = Adam(lr=2e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

ValueError: Exception encountered when calling layer 'tf_bert_model_6' (type TFBertModel).

Data of type <class 'keras.src.engine.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for input_ids.

Call arguments received by layer 'tf_bert_model_6' (type TFBertModel):
  • input_ids=["<KerasTensor: shape=(None, 128) dtype=int32 (created by layer 'input_ids_before')>", "<KerasTensor: shape=(None, 128) dtype=int32 (created by layer 'attention_masks_before')>"]
  • attention_mask=None
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False

In [ ]:
# Train model for before COVID
history_before = model.fit(
    [train_inputs_before, train_masks_before],
    train_labels_before,
    epochs=3,
    batch_size=32,
    validation_data=([test_inputs_before, test_masks_before], test_labels_before)
)

# Evaluate model for before COVID
predictions_before = model.predict([test_inputs_before, test_masks_before])
predicted_labels_before = np.argmax(predictions_before, axis=1)
true_labels_before = np.argmax(test_labels_before, axis=1)

print("Evaluation Report for Before COVID:")
print(classification_report(true_labels_before, predicted_labels_before))